In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv', parse_dates=['datetime'])
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv', parse_dates=['datetime'])
#sub = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

train.head(10)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train.head(10)

In [ ]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(16,12))
 
# temp, windspeed 의 히스토그램
train[['temp', 'windspeed']].hist(bins=20, ax=ax)

plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(20,8)

sns.barplot(data=train, x="season", y="count", ax=axes[0][0]) # 계졀벌 대여 갯수
sns.boxplot(data=train, x="workingday", y="count", ax=axes[0][1]) # Working day 기준 
sns.boxplot(data=train, x="holiday", y="count", ax=axes[1][0]) #Holiday 기준
sns.barplot(data=train, x="hour", y="count", ax=axes[1][1]) #시간대별 대여 갯수

In [ ]:
figure, ((ax1, ax2, ax3)) = plt.subplots(nrows=1, ncols=3)
figure.set_size_inches(20,8)

import seaborn as sns 
sns.barplot(data=train, x="year", y="count", ax=ax1) #년도 별 대여 (2012 > 2012)
sns.barplot(data=train, x="month", y="count", ax=ax2) # 월 별 대여 (주로 여름철에 대여량 증가)
sns.barplot(data=train, x="day", y="count", ax=ax3) # 일별 대여 ( 큰 차이의 추이는 없음 )

In [ ]:
def ym(mydt):
    return"{0}.{1}".format(mydt.year,mydt.month)
train['YM'] = train['datetime'].apply(ym)

fig = plt.figure(figsize=(18,6))
sns.barplot(data=train, x='YM', y="count") #2011년대비 12년에 대여수 증가

In [ ]:
fig = plt.figure(figsize=[20,20]) #Heat map
ax = sns.heatmap(train.corr(),annot=True,square=True)

In [ ]:
train.info()

In [ ]:
inttocat = ["season", "holiday", "workingday", "weather",'day','month','hour','year'] #as type으로 category 형변환 
for i in inttocat:
    train[i] = train[i].astype("category")
    test[i] = test[i].astype("category")

drop_columns = ['datetime','casual','registered']
train.drop(drop_columns, axis=1,inplace=True)
    
inttocat
train.info()

In [ ]:
features = ["season", "weather", "temp", "atemp", "humidity", "year", "hour", "month", "day", "holiday", "workingday"]
features

In [ ]:
X_train = train[features]

print(X_train.shape)
X_train.head(10)

In [ ]:
X_test = test[features]

print(X_test.shape)
X_test.head(10)

In [ ]:
label = 'count'
y_train = train[label]
y_train.head(10)

In [ ]:
#아웃라이어 제거 함수, count열값 - count열값 평균 < 3 * (count.std)
trainWithOutliers = train[np.abs(train["count"]-train["count"].mean())<= (3*train["count"].std())]
print(train.shape) #전
print(trainWithOutliers.shape) #후
print(X_train.shape)
print(X_test.shape)

In [ ]:
def rmsle(predicted_value, actual_value): #오류 값의 로그에 대한 RMSE
    predicted_value=np.array(predicted_value) # 예측값과 실제 값을 numpy 배열 형태로 변환
    actual_value=np.array(actual_value) # 예측값과 실제 값을 numpy 배열 형태로 변환
    
    log_predict=np.log(predicted_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
    log_actual=np.log(actual_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
    
    diff=log_predict-log_actual # 로그 변환한 예측값에서 로그 변환한 실제 값을 빼고 제곱
    diff=np.square(diff) # 로그 변환한 예측값에서 로그 변환한 실제 값을 빼고 제곱
    
    mean_diff=diff.mean() # 위에서 계산한 값의 평균
    
    score=np.sqrt(mean_diff) # 위에서 계산한 값에 루트
    return score

In [ ]:
from sklearn.metrics import make_scorer #1 Random Forest 방식 사용
rmsle_scorer=make_scorer(rmsle)
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from pandas import DataFrame

kfold=KFold(n_splits=10, shuffle=True, random_state=42) #K Fold 교차검증

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

model =RandomForestRegressor(n_estimators=100, n_jobs=-1,random_state=60)

%time score=cross_val_score(model, X_train, y_train,cv=kfold, scoring=rmsle_scorer )
print('Rmsle Random Forest Score:',score.mean())

In [ ]:
model.fit(X_train,y_train)

In [ ]:
prediction=model.predict(X_test)
prediction

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=4000, alpha=0.01);

y_train_log = np.log1p(y_train)
gbm.fit(X_train, y_train_log)

preds = gbm.predict(X_train)
score = rmsle(np.exp(y_train_log), np.exp(preds))
print("RMSLE Gradient Boost:", score)

In [ ]:
df1 = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
df1['count'] = prediction
df1.to_csv('test_submission.csv',index = False)
df1.head(10)